In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: misha-ml
Azure region: australiaeast
Subscription id: cf6550ec-d002-4ed0-b0df-f48bc54f2849
Resource group: misha-ml


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "AZUREML_CLUSTER"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
import os

# choose a name for your cluster
compute_name = os.environ.get(cluster_name, "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: cpu-cluster


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler 
ps = RandomParameterSampling( {
    "--C": choice(0.5, 1, 1.5),
    "--max_iter": choice(50, 100, 200)
    }
)

# Specify a Policy 
# Early termination policy. Check main metric every 2 runs, if falls outside of the top 10% - terminate)

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
project_folder = './'

src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     policy=policy,
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)


In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)



In [9]:
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
hyperdrive_run.wait_for_completion(show_output=True) 

RunId: HD_3ad4a0d4-ef2a-40ac-8540-7492dac4144f
Web View: https://ml.azure.com/runs/HD_3ad4a0d4-ef2a-40ac-8540-7492dac4144f?wsid=/subscriptions/cf6550ec-d002-4ed0-b0df-f48bc54f2849/resourcegroups/misha-ml/workspaces/misha-ml&tid=fbbe2706-bb0f-4c9e-a5a1-495087572df9

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-16T02:18:43.343641][API][INFO]Experiment created<END>\n""<START>[2022-02-16T02:18:44.077119][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"

Execution Summary
RunId: HD_3ad4a0d4-ef2a-40ac-8540-7492dac4144f
Web View: https://ml.azure.com/runs/HD_3ad4a0d4-ef2a-40ac-8540-7492dac4144f?wsid=/subscriptions/cf6550ec-d002-4ed0-b0df-f48bc54f2849/resourcegroups/misha-ml/workspaces/misha-ml&tid=fbbe2706-bb0f-4c9e-a5a1-495087572df9



{'runId': 'HD_3ad4a0d4-ef2a-40ac-8540-7492dac4144f',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-16T02:18:42.871249Z',
 'endTimeUtc': '2022-02-16T02:29:48.445319Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7ef80e96-0f0f-4055-ad22-ff3ae3f71109',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1064-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.37.0',
  'space_size': '9',
  'score': '0.9118866970156804',
  'best_child_run_id': 'HD_3ad4a0d4-ef2a-40ac-8540-7492dac4144f_7',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mishaml4127512191.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_3ad4a0d4-ef2a-40ac-8540-7492dac4144f/azure

In [11]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [12]:
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())

['--C', '1', '--max_iter', '100', '--C', '0.5', '--max_iter', '200']
['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


In [26]:
# register model 

model = best_run.register_model(
    model_name="sklearn_HP", model_path="./"
)
print(model.name, model.id, model.version, sep="\t")



sklearn_HP	sklearn_HP:1	1


## AutoML

In [35]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"


factory = TabularDatasetFactory()
data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = factory.from_delimited_files(data_path)


In [28]:
#from train import clean_data

# Use the clean_data function to clean your data.
#x, y = clean_data(ds)

# Split data 30% test, 70% train, random_state = 1
#from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
#train_data = x_train.merge(y_train, left_index=True, right_index=True)
#test_data = x_test.merge(y_test, left_index=True, right_index=True)


#train_dataset = Dataset.Tabular.register_pandas_dataframe(train_data,target='Auto_ML',name='train_dataset')
#test_dataset = Dataset.Tabular.register_pandas_dataframe(test_data,target='Auto_ML',name='train_dataset')


#label = "y"

In [36]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    #validation_data=test_data,
    n_cross_validations=2)

In [37]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output = False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_b6b23108-944b-4728-84b0-c2ac4bddd7b6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [38]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [39]:
# Picking the best model from Auto ML
best_run = remote_run.get_best_child()
print(best_run)

Run(Experiment: udacity-project,
Id: AutoML_b6b23108-944b-4728-84b0-c2ac4bddd7b6_30,
Type: azureml.scriptrun,
Status: Completed)


In [40]:
# Regester best model 
model = best_run.register_model(model_name='AutoML_model', model_path='./')
print(model.name, model.id, model.version, sep='\t')

AutoML_model	AutoML_model:1	1
